## requirements libs
### window
```
python-ffmpeg == 2.0.12
hidapi == 0.14.0.post4
pyserial == 3.5
```

## Setup project
``` bash
pip install -e .
```

In [ ]:
# steam example
from device import DeviceGroupsWin
from device import VideoFFmpeg

HID_VID = "534D"
HID_PID = "2109"
Serial_port_VID = "1A86"
Serial_port_PID = "7523"
device_info_list = DeviceGroupsWin.search_phycial_device(Serial_port_VID, Serial_port_PID, HID_VID, HID_PID)
video_path = device_info_list[0]['camera_path']
audio_path = device_info_list[0]['audio_path']
video_url = "rtp://192.168.100.66:5000"
VideoFFmpeg.start_stream(video_path, audio_path, video_url, "windows")

# Hot-Plugging Detection Feature

This section demonstrates the new hot-plugging detection functionality that monitors USB device connections and disconnections in real-time.

In [ ]:
# Import the hot-plugging detection functionality
from device import DeviceGroupsWin
import time
from datetime import datetime

# Device IDs for Openterface
HID_VID = "534D"
HID_PID = "2109"
Serial_port_VID = "1A86"
Serial_port_PID = "7523"

def hotplug_callback(event_data):
    """Callback function for device changes"""
    print(f"🔄 Device change detected at {event_data['timestamp']}")
    
    changes = event_data['changes_from_last']
    
    if changes['added_devices']:
        print(f"  ➕ Added: {len(changes['added_devices'])} devices")
        
    if changes['removed_devices']:
        print(f"  ➖ Removed: {len(changes['removed_devices'])} devices")
    
    current_count = len(event_data['current_devices'])
    print(f"  📊 Total devices now: {current_count}")

# Create hotplug monitor
monitor = DeviceGroupsWin.HotplugMonitor(
    serial_vid=Serial_port_VID,
    serial_pid=Serial_port_PID,
    hid_vid=HID_VID,
    hid_pid=HID_PID,
    poll_interval=3.0  # Check every 3 seconds
)

# Add callback and start monitoring
monitor.add_callback(hotplug_callback)
monitor.start_monitoring()

print("🟢 Hot-plug monitoring started!")
print("💡 Connect or disconnect Openterface devices to see real-time detection")
print("⏱️  Initial device scan completed")

In [ ]:
# Check current device state
current_state = monitor.get_current_state()
initial_state = monitor.get_initial_state()

if initial_state:
    print(f"📋 Initial State (captured at {initial_state['timestamp']}):")
    print(f"   Device count: {initial_state['device_count']}")
    
    for i, device in enumerate(initial_state['devices'], 1):
        print(f"   Device {i}:")
        if device.get('serial_port_path'):
            print(f"     Serial Port: {device['serial_port_path']}")
        if device.get('camera_path'):
            print(f"     Camera: Available")
        if device.get('audio_path'):
            print(f"     Audio: Available")
        if device.get('HID_path'):
            print(f"     HID: Available")
else:
    print("⚠️  No devices detected in initial scan")

print(f"\n💡 To stop monitoring, run: monitor.stop_monitoring()")

In [ ]:
# Manual snapshot comparison example
print("📸 Taking manual snapshots for comparison...")

# Take first snapshot
snapshot1 = DeviceGroupsWin.DeviceSnapshot(Serial_port_VID, Serial_port_PID, HID_VID, HID_PID)
print(f"   Snapshot 1 taken at {snapshot1.timestamp} - Found {len(snapshot1.devices)} devices")

# Wait a moment (you can connect/disconnect devices during this time)
print("⏳ Waiting 10 seconds... (connect/disconnect devices now)")
time.sleep(10)

# Take second snapshot
snapshot2 = DeviceGroupsWin.DeviceSnapshot(Serial_port_VID, Serial_port_PID, HID_VID, HID_PID)
print(f"   Snapshot 2 taken at {snapshot2.timestamp} - Found {len(snapshot2.devices)} devices")

# Compare snapshots
changes = snapshot2.compare_with(snapshot1)

print(f"\n🔍 Comparison Results:")
print(f"   Added devices: {len(changes['added_devices'])}")
print(f"   Removed devices: {len(changes['removed_devices'])}")
print(f"   Modified devices: {len(changes['modified_devices'])}")

if changes['added_devices']:
    print("   ➕ New devices:")
    for device in changes['added_devices']:
        print(f"      - Serial: {device.get('serial_port_path', 'N/A')}")

if changes['removed_devices']:
    print("   ➖ Removed devices:")
    for device in changes['removed_devices']:
        print(f"      - Serial: {device.get('serial_port_path', 'N/A')}")

In [ ]:
# Stop the hot-plug monitoring
monitor.stop_monitoring()
print("🛑 Hot-plug monitoring stopped")

## Serial port manager example
This example 

In [ ]:
#!/usr/bin/env python3
"""
Example usage of the SerialManager for Openterface Mini KVM
"""

import logging
import time
from serialPort.SerialManager import SerialManager
from serialPort.Ch9329 import CMD_GET_INFO, CMD_GET_PARA_CFG

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,  # Changed to DEBUG for more detailed output
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

def event_callback(event_type, data):
    """Callback function for serial port events"""
    print(f"Event: {event_type}, Data: {data}")

def data_ready_callback(data):
    """Callback function for received data"""
    print(f"Received data: {data.hex(' ')}")

def main():
    # Create SerialManager instance
    serial_manager = SerialManager()
    
    # Set callbacks
    serial_manager.set_event_callback(event_callback)
    serial_manager.set_data_ready_callback(data_ready_callback)
    
    # Specify the serial port path (modify this for your system)
    # Windows: "COM3", "COM4", etc.
    # Linux/Mac: "/dev/ttyUSB0", "/dev/ttyACM0", etc.
    port_path = "COM7"  # Change this to your actual port
    
    print(f"Attempting to connect to {port_path}...")
    
    # Connect to the device
    if serial_manager.connect(port_path):
        print(f"Successfully connected to: {serial_manager.get_port_name()}")
        
        try:
            # Keep the program running and periodically check device status
            while True:
                if serial_manager.is_ready():
                    print(f"Device Status:")
                    print(f"  Num Lock: {serial_manager.get_num_lock_state()}")
                    print(f"  Caps Lock: {serial_manager.get_caps_lock_state()}")
                    print(f"  Scroll Lock: {serial_manager.get_scroll_lock_state()}")
                    
                    # Send info command to refresh status
                    serial_manager.send_async_command(CMD_GET_INFO)
                    
                    time.sleep(5)
                else:
                    print("Device connection lost!")
                    break
                    
        except KeyboardInterrupt:
            print("\nShutting down...")
        finally:
            serial_manager.disconnect()
    else:
        print(f"Failed to connect to {port_path}")
        print("Please check:")
        print("1. The device is connected")
        print("2. The port path is correct")
        print("3. No other application is using the port")

if __name__ == "__main__":
    main()


In [ ]:
from serialPort.SerialManager import SerialManager
import time
import logging

# Enable debug logging to see the connection process
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

def data_ready_callback(data):
    """Callback function for received data"""
    print(f"Received data: {data.hex(' ')}")

def main():
    serial_manager = SerialManager()
    serial_manager.set_data_ready_callback(data_ready_callback)

    # Note: The baudrate parameter is now ignored - always uses 115200
    # If device is at 9600, it will be automatically reconfigured to 115200
    port_path = "COM7" 
    print(f"Connecting to {port_path}...")
    print("Note: Will always connect at 115200 baud. If device is at 9600, it will be reconfigured automatically.")
    
    if not serial_manager.connect(port_path):
        print(f"Failed to connect to {port_path}")
        print("Please check:")
        print("1. Device is connected")
        print("2. Port path is correct") 
        print("3. No other application is using the port")
        return
    
    print(f"Successfully connected to: {serial_manager.get_port_name()}")
    print(f"Final baudrate: {serial_manager.ser_port.baudrate if serial_manager.ser_port else 'Unknown'}")

    # Test keyboard commands
    print("\nSending keyboard commands...")
    time.sleep(0.5)
    
    # Open terminal (Ctrl+Alt+T for Linux, Win+R for Windows)
    serial_manager.send_key_combination("ctrl", "alt", "t")
    time.sleep(0.5)

    # Send simple command
    serial_manager.send_text("ls")
    time.sleep(1)

    # Press Enter
    serial_manager.send_key_press(0x28)  # Enter key code
    time.sleep(0.5)

    # Send echo command
    serial_manager.send_text("echo 'Hello, Openterface!'")
    time.sleep(2)

    serial_manager.send_key_press(0x28)  
    time.sleep(0.5)

    # Clear screen
    serial_manager.send_text("clear")
    time.sleep(0.5)

    # Press Enter again
    serial_manager.send_key_press(0x28)  # Enter key code
    time.sleep(0.5)

    serial_manager.send_text("exit")
    time.sleep(0.5)

    serial_manager.send_key_press(0x28)  # Enter key code
    time.sleep(0.5)

    print("Keyboard test completed!")
    serial_manager.disconnect()

if __name__ == "__main__":
    main()
